# Xác định event triggered workflow từ repo, issue hoặc pull request

- Xem xét kỹ Github Actions logs để đánh dấu các event đã xảy ra (push, pull request, issue creation, etc.)
- Logs tồn tại trong 90 ngày
    - Có thể điều chính tới 90d (public repo) hoặc 400d (private repo)
- Sử dụng ${{ github.event_name }} để xác định event type

VD:
```yaml
name: Identify Event Type
on: [push, pull_request, issues]

jobs:
    identify-event:
        runs-on: ubuntu-latest
        steps:
            - name: Identify Event
              run: echo "Event type is ${{ github.event_name }}"
```

# Mô tả workflow effects bằng cách đọc configuation file

- Mục đích chính để hiểu workflow effects và ảnh hưởng mà không chạy workflow
- Đọc file cấu hình workflow (`.github/workflows/*.yml`) để hiểu các bước, jobs, triggers, etc.
- Các hành động tương quan trong workflow với effects (build, test, deploy, etc.)

# Phân tích lỗi trong workflow run

- Kiểm tra thông báo lỗi và exit codes trong Actions log để xác định lỗi
- Review các lỗi thường gặp: syntax error, missing secrets, permission issues, etc.

# Xác định cách để truy cập vào workflow logs từ user interface

- Từ Actions tab trong repo -> chọn workflow run -> chọn job -> chọn step để xem logs

# Xác định cách truy cập Workflow logs từ Github REST API

- Sử dụng Github REST API endpoint GET `/repos/{owner}/{repo}/actions/runs/{run_id}/logs` để truy cập logs
- Authenticate API request với token hoặc personal access token
- Sử dụng `curl` hoặc các HTTP client để gửi request

# Cho phép step debug logging trong workflow

- Đặt secret `ACTIONS_STEP_DEBUG = true` trong repo để hiển thị debug logs cho mỗi step trong ***Workflow***

- Đặt secret `ACTIONS_RUNNER_DEBUG = true` để hiển thị debug logs cho ***Runner***

- Review logs để xác định lỗi và debug

# Mô tả cách làm thế nào để sử dụng các giá trị mặc định của environment trong workflow

- Sử dụng built-in variable như `GITHUB_SHA` và `GITHUB_REPOSITORY` để truy cập thông tin repo và commit
- Kết hợp những giá trị vào workflow steps để quản lý hành vi của workflow

VD:
```yaml
name: Tag Docker Image
on: push

jobs:
    docker-tag:
        runs-on: ubuntu-latest
        steps:
            - name: Checkout
              uses: actions/checkout@v2

            - name: Build and Tag Docker Image
              run: |
                docker build -t my-app:${{ github.sha }} .
                # Uses GITHUB_SHA to tag Docker image
```

# Syntax đúng khi passing custom environment variables vào workflow step

- Declare giá trị tùy chỉnh trong environment dưới dạng `env` trong workflow hoặc job level
- Truy cập chúng sử dụng `${{ env.VAR_NAME }}`

VD: Cách để lấy file logs sử dụng powershell
```ps1
$headers = @{
    "Accept"               = "application/vnd.github.v3+json"
    "Authorization"        = "Bearer $env:GITHUB_TOKEN"
    "X-GitHub-Api-Version" = "date-here"
}

$url = "https://api.github.com/repos/$env:GITHUB_REPOSITORY/actions/runs/$env:GITHUB_RUN_ID/logs"

response = Invoke-RestMethod -Uri $url -Method Get -Headers $headers
response | Format-List
```